In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline

C:\Users\Appzone-MLG\Anaconda3\envs\chest_xray_env\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# to display all the columns of the dataframe in the notebook
pd.pandas.set_option('display.max_columns', None)

In [10]:
# Reading the csv file 
# Here we will be using a subset of the original dataset

train = pd.read_csv('CheXpert-v1.0-small/train_sample.csv', nrows=2000)

In [11]:
train.head()

,path,no_finding,enlarged_cardiomediastinum,cardiomegaly,lung_opacity,lung_lesion,edema,consolidation,pneumonia,atelectasis,pneumothorax,pleural_effusion,pleural_other,fracture,no_of_labels
0,CheXpert-v1.0-small/train/patient00001/study1/...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,CheXpert-v1.0-small/train/patient00002/study2/...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
2,CheXpert-v1.0-small/train/patient00002/study1/...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
3,CheXpert-v1.0-small/train/patient00002/study1/...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0
4,CheXpert-v1.0-small/train/patient00003/study1/...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
# Columns of interest
train.columns

Index(['path', 'no_finding', 'enlarged_cardiomediastinum', 'cardiomegaly',
       'lung_opacity', 'lung_lesion', 'edema', 'consolidation', 'pneumonia',
       'atelectasis', 'pneumothorax', 'pleural_effusion', 'pleural_other',
       'fracture', 'no_of_labels'],
      dtype='object')

In [27]:
# Install Pillow and restart the Jupyter Server

# import sys
# import PIL
# from PIL import Image
# sys.modules['Image'] = Image 

# from IPython.display import display
# from PIL import Image

In [ ]:
# loading and preprocessing 
train_image = []
for i in tqdm(range(train.shape[0])):
    img = image.load_img(''+train['path'][i]+'',target_size=(400,400,3))
    img = image.img_to_array(img)
    img = img/255
    train_image.append(img)
X = np.array(train_image)


 98%|█████████████████████████████████████████████████████████████████████████▎ | 1954/2000 [00:17<00:01, 29.50it/s]


100%|██████████████████████████████████████████████████████████████████████████| 2000/2000 [00:19<00:00, 102.24it/s]


In [ ]:
X.shape

In [ ]:
plt.imshow(X[2])

In [ ]:
# loading the Multi-calssification binary classification label

y = np.array(train.drop(['path', 'no_of_labels'],axis=1))
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.1)

In [ ]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(5, 5), activation="relu", input_shape=(400,400,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters=64, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(25, activation='sigmoid'))

In [ ]:
# print our model summary

model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Model Training

model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=64)